In [1]:
import numpy as np
import math as m
import os
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

train_data={}
train_labels={}
locs={}
locs_2d={}
frames={}
images={}

In [2]:
from scipy.interpolate import griddata
from sklearn.preprocessing import scale


In [3]:
subjects=['bp',
    'ca',
    'cc',
    'de',
    'fp',
    'gc',
    'gf',
    'hh',
    'hl',
    'jc',
    'jf',
    'jm',
    'jp',
    'jt',
    'rh',
    'rr',
    'ug',
    'wc',
    'zt'
]

In [4]:
def cart2sph(x, y, z):
    """
    Transform Cartesian coordinates to spherical
    :param x: X coordinate
    :param y: Y coordinate
    :param z: Z coordinate
    :return: radius, elevation, azimuth
    """
    x2_y2 = x**2 + y**2
    r = m.sqrt(x2_y2 + z**2)                    # r
    elev = m.atan2(z, m.sqrt(x2_y2))            # Elevation
    az = m.atan2(y, x)                          # Azimuth
    return r, elev, az


def pol2cart(theta, rho):
    """
    Transform polar coordinates to Cartesian
    :param theta: angle value
    :param rho: radius value
    :return: X, Y
    """
    return rho * m.cos(theta), rho * m.sin(theta)

In [5]:
def azim_proj(pos):
    """
    Computes the Azimuthal Equidistant Projection of input point in 3D Cartesian Coordinates.
    Imagine a plane being placed against (tangent to) a globe. If
    a light source inside the globe projects the graticule onto
    the plane the result would be a planar, or azimuthal, map
    projection.

    :param pos: position in 3D Cartesian coordinates
    :return: projected coordinates using Azimuthal Equidistant Projection
    """
    [r, elev, az] = cart2sph(pos[0], pos[1], pos[2])
    return pol2cart(az, m.pi / 2 - elev)

In [6]:
path=os.path.join("..","epoched_data/motor_basic")
for sub in subjects:
    train_data[sub]=np.load(path+"/"+sub+"_mot_epochs.npz")['X']
    train_labels[sub]=np.load(path+"/"+sub+"_mot_epochs.npz")['y']

In [7]:
path_locs=os.path.join("..","motor_basic_npz_raw")
for sub in subjects:
    locs[sub]=np.load(path_locs+"/"+sub+"_mot.npz")['electrodes']

In [9]:
def get_averages(f,Y):
    low_alpha_range=(7,12)
    high_alpha_range=(10,13)
    beta_range=(14,25)
    low_gamma_range=(26,35)
    high_gamma_range=(36,70)
    beta = Y[(f>beta_range[0]) & (f<=beta_range[1])].mean()/100
    low_alpha = Y[(f>low_alpha_range[0]) & (f<=low_alpha_range[1])].mean()/100
    high_alpha = Y[(f>high_alpha_range[0]) & (f<=high_alpha_range[1])].mean()/100
    low_gamma = Y[(f>low_gamma_range[0]) & (f<=low_gamma_range[1])].mean()/100
    high_gamma = Y[(f>high_gamma_range[0]) & (f<=high_gamma_range[1])].mean()/100
    return low_alpha,high_alpha,beta,low_gamma,high_gamma

In [10]:
for sub in subjects:
    n_electrodes=locs[sub].shape[0]
    locs_2d[sub]=np.empty((n_electrodes,2))
    for i in range (n_electrodes):
        locs_2d[sub][i]=azim_proj(locs[sub][i])

In [11]:
def get_fft(snippet):
    Fs = 1000;  # sampling rate
    #Ts = len(snippet)/Fs/Fs; # sampling interval
    snippet_time = len(snippet)/Fs
    Ts = 1.0/Fs; # sampling interval
    t = np.arange(0,snippet_time,Ts) # time vector

    # ff = 5;   # frequency of the signal
    # y = np.sin(2*np.pi*ff*t)
    y = snippet
#     print('Ts: ',Ts)
#     print(t)
#     print(y.shape)
    n = len(y) # length of the signal
    k = np.arange(n)
    T = n/Fs
    frq = k/T # two sides frequency range
    frq = frq[range(n//2)] # one side frequency range

    Y = np.fft.fft(y)/n # fft computing and normalization
    Y = Y[range(n//2)]
    #Added in: (To remove bias.)
    #Y[0] = 0
    return frq,abs(Y)
#f,Y = get_fft(np.hanning(len(snippet))*snippet)

In [12]:
Fs=1000
image_size = 64
frame_duration = 3.0
overlap=0
frame_length = Fs * frame_duration

In [13]:
for sub in subjects:
    frames[sub] = []
    n_electrodes=train_data[sub].shape[1]
    for trial in range(train_data[sub].shape[0]):
        frame=[]
        for i in range (n_electrodes):
            snippet = train_data[sub][trial,i,:]
            f,Y =  get_fft(snippet)
            la, ha, b, lg, hg = get_averages(f,Y)
            frame.append([la, ha, b, lg, hg])
        frames[sub].append(frame)
    frames[sub]=np.array(frames[sub])

In [14]:
frames['bp']

array([[[0.61156519, 0.67253133, 0.37972937, 0.21930585, 0.12005414],
        [0.4780272 , 0.44575872, 0.30019389, 0.1828456 , 0.11578361],
        [1.19548536, 0.81989393, 0.43793013, 0.14342559, 0.09257395],
        ...,
        [0.89934715, 0.87293335, 0.37064674, 0.17643609, 0.08866591],
        [1.05559722, 0.99615298, 0.45273453, 0.18923188, 0.11981511],
        [1.45937131, 1.20889451, 0.4218755 , 0.23669631, 0.1265039 ]],

       [[0.56189289, 0.49829043, 0.31968641, 0.19928478, 0.11942025],
        [0.75526592, 0.57479568, 0.30391421, 0.166267  , 0.10026313],
        [0.76541458, 0.7832127 , 0.30195337, 0.18460209, 0.09363034],
        ...,
        [0.89830231, 0.76261016, 0.47665726, 0.18492275, 0.10504374],
        [1.09172056, 0.87285284, 0.31154663, 0.20447486, 0.12318371],
        [1.04732371, 0.71303282, 0.43224054, 0.20208507, 0.13254414]],

       [[0.80117161, 0.58382678, 0.41379011, 0.18826627, 0.14075787],
        [0.76543972, 0.66783531, 0.32579026, 0.16412918, 0.1

In [15]:
def gen_images(locs, features, n_gridpoints, normalize=True,
               augment=False, pca=False, std_mult=0.1, n_components=2, edgeless=False):
    """
    Generates EEG images given electrode locations in 2D space and multiple feature values for each electrode

    :param locs: An array with shape [n_electrodes, 2] containing X, Y
                        coordinates for each electrode.
    :param features: Feature matrix as [n_samples, n_features]
                                Features are as columns.
                                Features corresponding to each frequency band are concatenated.
                                (alpha1, alpha2, ..., beta1, beta2,...)
    :param n_gridpoints: Number of pixels in the output images
    :param normalize:   Flag for whether to normalize each band over all samples
    :param augment:     Flag for generating augmented images
    :param pca:         Flag for PCA based data augmentation
    :param std_mult     Multiplier for std of added noise
    :param n_components: Number of components in PCA to retain for augmentation
    :param edgeless:    If True generates edgeless images by adding artificial channels
                        at four corners of the image with value = 0 (default=False).
    :return:            Tensor of size [samples, colors, W, H] containing generated
                        images.
    """
    feat_array_temp = []
    nElectrodes = locs.shape[0]     # Number of electrodes
    # Test whether the feature vector length is divisible by number of electrodes
    assert features.shape[1] % nElectrodes == 0
    n_colors = features.shape[1] // nElectrodes
    for c in range(int(n_colors)):
        feat_array_temp.append(features[:, c * nElectrodes : nElectrodes * (c+1)])
    nSamples = features.shape[0]
    # Interpolate the values
    grid_x, grid_y = np.mgrid[
                     min(locs[:, 0]):max(locs[:, 0]):n_gridpoints*1j,
                     min(locs[:, 1]):max(locs[:, 1]):n_gridpoints*1j
                     ]
    temp_interp = []
    for c in range(n_colors):
        temp_interp.append(np.zeros([nSamples, n_gridpoints, n_gridpoints]))
    # Generate edgeless images
    if edgeless:
        min_x, min_y = np.min(locs, axis=0)
        max_x, max_y = np.max(locs, axis=0)
        locs = np.append(locs, np.array([[min_x, min_y], [min_x, max_y],[max_x, min_y],[max_x, max_y]]),axis=0)
        for c in range(n_colors):
            feat_array_temp[c] = np.append(feat_array_temp[c], np.zeros((nSamples, 4)), axis=1)
    # Interpolating
    for i in range(nSamples):
        for c in range(n_colors):
            temp_interp[c][i, :, :] = griddata(locs, feat_array_temp[c][i, :], (grid_x, grid_y),
                                    method='cubic', fill_value=np.nan)
        print('Interpolating {0}/{1}\r'.format(i+1, nSamples), end='\r')
    # Normalizing
    for c in range(n_colors):
        if normalize:
            temp_interp[c][~np.isnan(temp_interp[c])] = \
                scale(temp_interp[c][~np.isnan(temp_interp[c])])
        temp_interp[c] = np.nan_to_num(temp_interp[c])
    return np.swapaxes(np.asarray(temp_interp), 0, 1)     # swap axes to have [samples, colors, W, H]

In [16]:
for sub in subjects:
    temp=frames[sub].reshape(len(frames[sub]),locs[sub].shape[0]*5)
    images[sub]=gen_images(locs_2d[sub],temp,image_size,normalize=False)
    images[sub]=np.swapaxes(images[sub],1,3)

In [24]:
for sub in subjects:
    path=os.getcwd()
    path=os.path.join(path,sub)
    os.makedirs(path)
    for i in range (len(images[sub])):
        plt.imsave(path+"/"+sub+"_"+str(i)+"_la.png",images[sub][i,:,:,0])
        plt.imsave(path+"/"+sub+"_"+str(i)+"_ha.png",images[sub][i,:,:,1])
        plt.imsave(path+"/"+sub+"_"+str(i)+"_b.png",images[sub][i,:,:,2])
        plt.imsave(path+"/"+sub+"_"+str(i)+"_lg.png",images[sub][i,:,:,3])
        plt.imsave(path+"/"+sub+"_"+str(i)+"_hg.png",images[sub][i,:,:,4])

In [25]:
X=images['ca']
y=train_labels['ca']
for i in range (len(y)):
    if y[i]>0:
        y[i]=y[i]-10


In [26]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# input image dimensions
img_rows, img_cols = 64,64

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

input_shape = (img_rows, img_cols, 5)

import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

batch_size = 16
num_classes =3
epochs = 100

# convert class vectors to binary class matrices
y_train = tensorflow.keras.utils.to_categorical(y_train, num_classes)
y_test = tensorflow.keras.utils.to_categorical(y_test, num_classes)


model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',input_shape=input_shape))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(10))
model.add(Activation('relu'))
# model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = tensorflow.keras.optimizers.RMSprop(lr=0.001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
#x_train /= 255
#x_test /= 255


model.fit(x_train, y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(x_test, y_test),
        shuffle=True,verbose=2)


x_train shape: (90, 64, 64, 5)
90 train samples
31 test samples
Epoch 1/100
6/6 - 1s - loss: 2.5331 - accuracy: 0.4444 - val_loss: 0.9704 - val_accuracy: 0.5806
Epoch 2/100
6/6 - 0s - loss: 1.0565 - accuracy: 0.4778 - val_loss: 0.9550 - val_accuracy: 0.5806
Epoch 3/100
6/6 - 0s - loss: 1.0546 - accuracy: 0.4778 - val_loss: 1.0128 - val_accuracy: 0.5806
Epoch 4/100
6/6 - 0s - loss: 0.9979 - accuracy: 0.4778 - val_loss: 1.6260 - val_accuracy: 0.2258
Epoch 5/100
6/6 - 0s - loss: 1.1420 - accuracy: 0.4111 - val_loss: 1.0421 - val_accuracy: 0.5484
Epoch 6/100
6/6 - 0s - loss: 1.0108 - accuracy: 0.4889 - val_loss: 0.9334 - val_accuracy: 0.5806
Epoch 7/100
6/6 - 0s - loss: 0.9748 - accuracy: 0.4778 - val_loss: 1.0009 - val_accuracy: 0.6129
Epoch 8/100
6/6 - 0s - loss: 0.8976 - accuracy: 0.5111 - val_loss: 0.9010 - val_accuracy: 0.5806
Epoch 9/100
6/6 - 0s - loss: 0.9002 - accuracy: 0.5000 - val_loss: 0.9917 - val_accuracy: 0.5484
Epoch 10/100
6/6 - 0s - loss: 1.0055 - accuracy: 0.5778 - val_l